In [3]:
import pandas as pd
import numpy as np
import math
# import missingno as mn
# import matplotlib.pyplot as plt
# import seaborn as sn
import pickle
from geopy.geocoders import Nominatim

#read in the pubs dataset already processed with distance to their nearest tube line attached
#easiest thing is probably to start adding total travel time to nearest node point of a different line
#again can cover that later, for now just take the closest one
#orrrr just treat them as independant pubs and double the length of your data, it wouldn't add much processing time


def get_coords_for_address(numentries):
    d= {}
    # z = 1
#     while z <= numentries:
    for z in range(numentries):
        #user_input = input
        d["user_input{0}".format(z)] = input("What's your address/postcode? ")
        #type(name)
        #d["user_input{0}".format(z)]= "London"
        d["user_input_geocode{0}".format(z)]  = geolocator.geocode(d["user_input{0}".format(z)])
        try:
            d["user_input_latitude{0}".format(z)] = d["user_input_geocode{0}".format(z)].latitude
            d["user_input_longitude{0}".format(z)] = d["user_input_geocode{0}".format(z)].longitude
        except:
            print("This failed! Please try again.")
        z += 1


In [4]:

geolocator = Nominatim(user_agent= "GoogleV3")
numentries = int(input("How many addresses?"))
get_coords_for_address(numentries)


How many addresses?1
What's your address/postcode? e149eg
This failed! Please try again.


In [ ]:
def get_coords_for_address(numentries):
    d= {}
    # z = 1
    # while z <= numentries:
    for z in range(numentries):
        #user_input = input
        result = None
        while result is None:
            d["user_input{0}".format(z)] = input("What's your address? ")
            #type(name)
            #d["user_input{0}".format(z)]= "London"
            d["user_input_geocode{0}".format(z)]  = geolocator.geocode(d["user_input{0}".format(z)])
            result = d["user_input_geocode{0}".format(z)]

            try:
                d["user_input_latitude{0}".format(z)] = d["user_input_geocode{0}".format(z)].latitude
                d["user_input_longitude{0}".format(z)] = d["user_input_geocode{0}".format(z)].longitude
            except:
                print("Google can't locate that address! Please try again.")
            z += 1

geolocator = Nominatim(user_agent= "GoogleV3")
numentries = int(input("How many addresses?"))
get_coords_for_address(numentries)


In [10]:

print(geolocator.geocode('e149eg'))

None


In [12]:
geolocator.geocode('london')

Location(London, Greater London, England, SW1A 2DX, United Kingdom, (51.5073219, -0.1276474, 0.0))